In [1]:
import json
import numpy as np
import pandas as pd
import psycopg2

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 500)

## Import and format the data

First, let's import the data.

In [2]:
complaints = pd.read_csv("data/complaints_11_17.txt", sep="\t")
complaints.rename(columns={"(SELECTSUM(NVL(BACKPAY,0)+NVL(FRONTPAY,0)+NVL(INTEREST,0)+NVL(LIQUIDATED_DAMAGES,0)+NVL(NON_PEC_COMP_DAMAGES,0)+NVL(PEC_COMP_DAMAGES,0)+NVL(PUNITIVE_DAMAGES,0)+NVL(COSTS_AND_FEES,0))FROMLEG_BENEFITLBWHERELB.LEGCASE_CASE_SEQ=AL7.CASE_SEQ)":"litigation_monetary_benefits"}, inplace=True)
complaints.info()

/home/jyerardi/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1,5,15,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3443516 entries, 0 to 3443515
Data columns (total 28 columns):
FISCAL_YEAR                     object
CHARGE_INQUIRY_SEQ              object
STATE_CODE                      object
NUMBER_OF_EMPLOYEES_CODE        object
NUMBER_OF_EMPLOYEES             object
NAICS_CODE                      object
NAICS_DESCRIPTION               object
INSTITUTION_TYPE_CODE           object
INSTITUTION_TYPE                object
DATE_OF_BIRTH                   object
SEX_CODE                        object
DATE_FIRST_OFFICE               object
CLOSURE_DATE                    object
CLOSURE_CODE                    object
CLOSURE_ACTION                  object
TOTAL_BENEFIT_AMOUNT            object
STATUTE_CODE                    object
STATUTE                         object
BASIS_CODE                      object
BASIS                           object
ISSUE_CODE                      object
ISSUE                           object
COURT_FILING_DATE            

Eliminate capital letters and spaces in column headers.

In [3]:
complaints.columns = map(str.lower, complaints.columns)
complaints.columns = complaints.columns.str.replace(" ","_")

Drop repetitive header rows.

In [4]:
complaints = complaints[complaints["charge_inquiry_seq"] != "CHARGE_INQUIRY_SEQ"]

Convert the dataframe's columns to their appropriate data types.

In [5]:
complaints["total_benefit_amount"] = pd.to_numeric(complaints["total_benefit_amount"], errors="coerce", downcast='float')
complaints["litigation_monetary_benefits"] = pd.to_numeric(complaints["litigation_monetary_benefits"], errors="coerce", downcast='float')

In [6]:
complaints.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3443510 entries, 0 to 3443515
Data columns (total 28 columns):
fiscal_year                     object
charge_inquiry_seq              object
state_code                      object
number_of_employees_code        object
number_of_employees             object
naics_code                      object
naics_description               object
institution_type_code           object
institution_type                object
date_of_birth                   object
sex_code                        object
date_first_office               object
closure_date                    object
closure_code                    object
closure_action                  object
total_benefit_amount            float32
statute_code                    object
statute                         object
basis_code                      object
basis                           object
issue_code                      object
issue                           object
court_filing_date           

We'll want to run groupby queries in our analysis. Python ignores NaN values in such queries so we need to fill in those values with a placeholder.

In [7]:
complaints.fillna(0, inplace=True)
complaints.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3443510 entries, 0 to 3443515
Data columns (total 28 columns):
fiscal_year                     object
charge_inquiry_seq              object
state_code                      object
number_of_employees_code        object
number_of_employees             object
naics_code                      object
naics_description               object
institution_type_code           object
institution_type                object
date_of_birth                   object
sex_code                        object
date_first_office               object
closure_date                    object
closure_code                    object
closure_action                  object
total_benefit_amount            float32
statute_code                    object
statute                         object
basis_code                      object
basis                           object
issue_code                      object
issue                           object
court_filing_date           

## How many charges and cases are filed by year?

Now, let's analyze the data. We can start by seeing how many charges are filed by year.

In [8]:
charges_by_year = complaints.groupby("fiscal_year").size().reset_index(name="charges")
charges_by_year

,fiscal_year,charges
0,0,3
1,,1
2,FY2011,480833
3,FY2012,485451
4,FY2013,486186
5,FY2014,496966
6,FY2015,491092
7,FY2016,514604
8,FY2017,488374


And how many cases are filed by year?

In [9]:
cases_by_year = complaints.groupby("fiscal_year").agg({"charge_inquiry_seq": pd.Series.nunique})
cases_by_year

,charge_inquiry_seq
fiscal_year,
0,1
,1
FY2011,163274
FY2012,166765
FY2013,160906
FY2014,161383
FY2015,155320
FY2016,150822
FY2017,173986


## How are different types of cases resolved?

Next, we'd like to know how different types of cases ("bases") are resolved. To do so, we'll need to group by the basis and closure action.

In [10]:
resolution_by_basis = pd.pivot_table(complaints, index=["basis", "closure_action"], values="charge_inquiry_seq", aggfunc=len)
resolution_by_basis.rename(columns={"charge_inquiry_seq": "charges"}, inplace=True)

In [11]:
resolution_by_basis.head()

charges
basis closure_action                  
0     0                              4
Age   0                          11210
      ADEA Sect. 7(D) Closure     1134
      Administrative Closure      2799
      CP Failed To Cooperate       623

Now we'll reset the index, calculate the total number of charges per basis and calculate the percentage of each basis that receives a particular closure action.

In [12]:
resolution_by_basis.reset_index(inplace=True)
resolution_by_basis["total_by_basis"] = resolution_by_basis["charges"].groupby(resolution_by_basis["basis"]).transform(sum)
resolution_by_basis["percent_of_charges"] = resolution_by_basis["charges"] / resolution_by_basis["total_by_basis"]
resolution_by_basis["rank"] = resolution_by_basis.groupby(["basis"])["percent_of_charges"].rank(method="min", ascending=False).astype(int)

In [13]:
resolution_by_basis.head(25)

,basis,closure_action,charges,total_by_basis,percent_of_charges,rank
0,0,0,4,4,1.000000,1
1,Age,0,11210,346135,0.032386,6
2,Age,ADEA Sect. 7(D) Closure,1134,346135,0.003276,13
3,Age,Administrative Closure,2799,346135,0.008086,10
4,Age,CP Failed To Cooperate,623,346135,0.001800,14
5,Age,CP Failed To Respond To 30-Day Letter,422,346135,0.001219,15
6,Age,CP Filed Suit,3273,346135,0.009456,9
7,Age,CP Refused Full Relief,20,346135,0.000058,22
8,Age,CP Withdrawal - No Ben.,9463,346135,0.027339,7
9,Age,Case Settled By Legal Unit,44,346135,0.000127,18


## How do the bases and outcomes of complaints differ by the complainant's gender?

First, connect to the database.

In [14]:
with open("config.json") as f:
    conf = json.load(f)

In [15]:
conn_str = "host={} dbname={} user={} password={}".format(conf["host"], conf["database"], conf["user"], conf["password"])

In [16]:
conn = psycopg2.connect(conn_str)

Query the database to return the basis and closure action of all charges where the complainant was a woman.

In [17]:
female_bases = pd.read_sql("""select basis, closure_action, count(*) as charges
                           from eeoc_complaints_11_17
                           where sex_code = 'F'
                           group by basis, closure_action
                           order by basis, closure_action asc""",
                           con=conn)

Let's rank the bases and closure actions.

In [18]:
female_bases["charges_per_basis"] = female_bases["charges"].groupby(female_bases["basis"]).transform(sum)
female_bases["percent_of_basis"] = female_bases["charges"] / female_bases["charges_per_basis"]
female_bases["basis_rank"] = female_bases.groupby(["basis"])["percent_of_basis"].rank(method="min", ascending=False).astype(int)
female_bases["percent_of_total"] = female_bases["charges_per_basis"] / female_bases["charges"].sum()
female_bases["total_rank"] = female_bases["percent_of_total"].rank(method="dense", ascending=False).astype(int)
female_bases.sort_values(["percent_of_total", "percent_of_basis"], ascending=False)

,basis,closure_action,charges,charges_per_basis,percent_of_basis,basis_rank,percent_of_total,total_rank
947,Retaliation,No Cause Finding Issued,257952,479993,0.537408,1,0.266242,1
955,Retaliation,None,57953,479993,0.120737,2,0.266242,1
949,Retaliation,NRTS Issued At CP Request,51462,479993,0.107214,3,0.266242,1
954,Retaliation,Withdrawal With Benefits,38736,479993,0.080701,4,0.266242,1
952,Retaliation,Settlement With Benefits,35067,479993,0.073057,5,0.266242,1
944,Retaliation,CP Withdrawal - No Ben.,10640,479993,0.022167,6,0.266242,1
948,Retaliation,No Jurisdiction,8515,479993,0.017740,7,0.266242,1
939,Retaliation,Conciliation Failure,5977,479993,0.012452,8,0.266242,1
936,Retaliation,Administrative Closure,4112,479993,0.008567,9,0.266242,1
942,Retaliation,CP Filed Suit,3836,479993,0.007992,10,0.266242,1


And now let's do the same for all charges where the complainant was a man.

In [19]:
male_bases = pd.read_sql("""select basis, closure_action, count(*) as charges
                           from eeoc_complaints_11_17
                           where sex_code = 'M'
                           group by basis, closure_action
                           order by basis, closure_action asc""",
                           con=conn)

In [20]:
male_bases["charges_per_basis"] = male_bases["charges"].groupby(male_bases["basis"]).transform(sum)
male_bases["percent_of_basis"] = male_bases["charges"] / male_bases["charges_per_basis"]
male_bases["basis_rank"] = male_bases.groupby(["basis"])["percent_of_basis"].rank(method="min", ascending=False).astype(int)
male_bases["percent_of_total"] = male_bases["charges_per_basis"] / male_bases["charges"].sum()
male_bases["total_rank"] = male_bases["percent_of_total"].rank(method="dense", ascending=False).astype(int)
male_bases.sort_values(["percent_of_total", "percent_of_basis"], ascending=False)

,basis,closure_action,charges,charges_per_basis,percent_of_basis,basis_rank,percent_of_total,total_rank
954,Retaliation,No Cause Finding Issued,168768,299327,0.563825,1,0.242869,1
962,Retaliation,None,33824,299327,0.113000,2,0.242869,1
956,Retaliation,NRTS Issued At CP Request,30458,299327,0.101755,3,0.242869,1
961,Retaliation,Withdrawal With Benefits,20751,299327,0.069326,4,0.242869,1
959,Retaliation,Settlement With Benefits,19228,299327,0.064237,5,0.242869,1
955,Retaliation,No Jurisdiction,8199,299327,0.027391,6,0.242869,1
951,Retaliation,CP Withdrawal - No Ben.,6476,299327,0.021635,7,0.242869,1
946,Retaliation,Conciliation Failure,3596,299327,0.012014,8,0.242869,1
943,Retaliation,Administrative Closure,2496,299327,0.008339,9,0.242869,1
949,Retaliation,CP Filed Suit,2252,299327,0.007524,10,0.242869,1


What's the number one closure action for each basis?

In [21]:
female_bases[female_bases["basis_rank"] == 1]

,basis,closure_action,charges,charges_per_basis,percent_of_basis,basis_rank,percent_of_total,total_rank
12,Age,No Cause Finding Issued,98068,162544,0.603332,1,0.090160,4
27,Alcoholism,No Cause Finding Issued,564,983,0.573754,1,0.000545,56
41,Allergies,No Cause Finding Issued,1127,1885,0.597878,1,0.001046,48
49,Alzheimers,No Cause Finding Issued,24,43,0.558140,1,0.000024,76
61,Asthma,No Cause Finding Issued,2574,4434,0.580514,1,0.002459,34
71,Autism,No Cause Finding Issued,207,371,0.557951,1,0.000206,67
84,Blood (Other),No Cause Finding Issued,1408,2493,0.564781,1,0.001383,44
91,Brain/Head Impairment,Settlement With Benefits,2,2,1.000000,1,0.000001,79
98,Brain/Head Injury (Traumatic),No Cause Finding Issued,1337,2520,0.530556,1,0.001398,43
114,Cancer,No Cause Finding Issued,4597,10135,0.453577,1,0.005622,20


In [22]:
male_bases[male_bases["basis_rank"] == 1]

,basis,closure_action,charges,charges_per_basis,percent_of_basis,basis_rank,percent_of_total,total_rank
12,Age,No Cause Finding Issued,82911,138758,0.597522,1,0.112586,3
26,Alcoholism,No Cause Finding Issued,1116,1975,0.565063,1,0.001602,43
39,Allergies,No Cause Finding Issued,405,632,0.640823,1,0.000513,64
46,Alzheimers,No Cause Finding Issued,29,34,0.852941,1,0.000028,79
56,Asthma,No Cause Finding Issued,941,1713,0.549329,1,0.001390,45
67,Autism,No Cause Finding Issued,532,993,0.535750,1,0.000806,60
81,Blood (Other),No Cause Finding Issued,898,1554,0.577864,1,0.001261,48
94,Brain/Head Injury (Traumatic),No Cause Finding Issued,1427,2581,0.552886,1,0.002094,38
108,Cancer,No Cause Finding Issued,2354,5449,0.432006,1,0.004421,29
120,Cerebral Palsy,No Cause Finding Issued,374,735,0.508844,1,0.000596,62


Export the data to Excel for further analysis.

In [23]:
writer = pd.ExcelWriter("data/query_exports/bases_by_gender.xlsx")
female_bases.to_excel(writer, "bases_by_gender_female", startcol=0, index=False)
male_bases.to_excel(writer, "bases_by_gender_male", startcol=0, index=False)
writer.save()

## How do the issues and outcomes of complaints differ by the complainant's gender?

Query the database to return the issue and closure action of all charges where the complainant was a woman.

In [24]:
female_issues = pd.read_sql("""select issue, closure_action, count(*) as charges
                           from eeoc_complaints_11_17
                           where sex_code = 'F'
                           group by issue, closure_action
                           order by issue, closure_action asc""",
                           con=conn)

Let's rank the issues and closure actions.

In [25]:
female_issues["charges_per_issue"] = female_issues["charges"].groupby(female_issues["issue"]).transform(sum)
female_issues["percent_of_issue"] = female_issues["charges"] / female_issues["charges_per_issue"]
female_issues["issue_rank"] = female_issues.groupby(["issue"])["percent_of_issue"].rank(method="min", ascending=False).astype(int)
female_issues["percent_of_total"] = female_issues["charges_per_issue"] / female_issues["charges"].sum()
female_issues["total_rank"] = female_issues["percent_of_total"].rank(method="dense", ascending=False).astype(int)
female_issues.sort_values(["percent_of_total", "percent_of_issue"], ascending=False)

,issue,closure_action,charges,charges_per_issue,percent_of_issue,issue_rank,percent_of_total,total_rank
146,Discharge,No Cause Finding Issued,274708,480124,0.572161,1,0.266315,1
154,Discharge,None,50489,480124,0.105158,2,0.266315,1
148,Discharge,NRTS Issued At CP Request,42690,480124,0.088915,3,0.266315,1
153,Discharge,Withdrawal With Benefits,36860,480124,0.076772,4,0.266315,1
151,Discharge,Settlement With Benefits,36193,480124,0.075383,5,0.266315,1
147,Discharge,No Jurisdiction,10940,480124,0.022786,6,0.266315,1
143,Discharge,CP Withdrawal - No Ben.,9382,480124,0.019541,7,0.266315,1
138,Discharge,Conciliation Failure,5395,480124,0.011237,8,0.266315,1
152,Discharge,Successful Conciliation,3735,480124,0.007779,9,0.266315,1
135,Discharge,Administrative Closure,3564,480124,0.007423,10,0.266315,1


And now let's do the same for all charges where the complainant was a man.

In [26]:
male_issues = pd.read_sql("""select issue, closure_action, count(*) as charges
                           from eeoc_complaints_11_17
                           where sex_code = 'M'
                           group by issue, closure_action
                           order by issue, closure_action asc""",
                           con=conn)

In [27]:
male_issues["charges_per_issue"] = male_issues["charges"].groupby(male_issues["issue"]).transform(sum)
male_issues["percent_of_issue"] = male_issues["charges"] / male_issues["charges_per_issue"]
male_issues["issue_rank"] = male_issues.groupby(["issue"])["percent_of_issue"].rank(method="min", ascending=False).astype(int)
male_issues["percent_of_total"] = male_issues["charges_per_issue"] / male_issues["charges"].sum()
male_issues["total_rank"] = male_issues["percent_of_total"].rank(method="dense", ascending=False).astype(int)
male_issues.sort_values(["percent_of_total", "percent_of_issue"], ascending=False)

,issue,closure_action,charges,charges_per_issue,percent_of_issue,issue_rank,percent_of_total,total_rank
143,Discharge,No Cause Finding Issued,219563,366917,0.598400,1,0.297710,1
151,Discharge,None,35920,366917,0.097897,2,0.297710,1
145,Discharge,NRTS Issued At CP Request,30589,366917,0.083368,3,0.297710,1
150,Discharge,Withdrawal With Benefits,25726,366917,0.070114,4,0.297710,1
148,Discharge,Settlement With Benefits,25112,366917,0.068441,5,0.297710,1
144,Discharge,No Jurisdiction,9625,366917,0.026232,6,0.297710,1
140,Discharge,CP Withdrawal - No Ben.,7069,366917,0.019266,7,0.297710,1
135,Discharge,Conciliation Failure,3716,366917,0.010128,8,0.297710,1
132,Discharge,Administrative Closure,2873,366917,0.007830,9,0.297710,1
138,Discharge,CP Filed Suit,2330,366917,0.006350,10,0.297710,1


What's the number one closure action for each issue?

In [28]:
female_issues[female_issues["issue_rank"] == 1]

,issue,closure_action,charges,charges_per_issue,percent_of_issue,issue_rank,percent_of_total,total_rank
2,Advertising,No Cause Finding Issued,33,124,0.266129,1,0.000069,46
13,Apprenticeship,No Cause Finding Issued,77,190,0.405263,1,0.000105,44
30,Assignment,No Cause Finding Issued,24940,43409,0.574535,1,0.024078,11
48,Benefits,No Cause Finding Issued,6825,12410,0.549960,1,0.006884,18
63,Benefits-Insurance,No Cause Finding Issued,1936,3491,0.554569,1,0.001936,22
75,Benefits-Retirement/Pension,No Cause Finding Issued,977,1453,0.672402,1,0.000806,31
88,Breach of Confidentiality,No Cause Finding Issued,1059,1979,0.535119,1,0.001098,27
106,Constructive Discharge,No Cause Finding Issued,37102,65585,0.565709,1,0.036379,7
125,Demotion,No Cause Finding Issued,15219,28937,0.525936,1,0.016051,15
146,Discharge,No Cause Finding Issued,274708,480124,0.572161,1,0.266315,1


In [29]:
male_issues[male_issues["issue_rank"] == 1]

,issue,closure_action,charges,charges_per_issue,percent_of_issue,issue_rank,percent_of_total,total_rank
3,Advertising,No Cause Finding Issued,64,285,0.224561,1,0.000231,42
12,Apprenticeship,No Cause Finding Issued,107,180,0.594444,1,0.000146,45
29,Assignment,No Cause Finding Issued,16475,29097,0.566210,1,0.023609,12
46,Benefits,No Cause Finding Issued,5042,8627,0.584444,1,0.007000,18
62,Benefits-Insurance,No Cause Finding Issued,1572,2776,0.566282,1,0.002252,23
75,Benefits-Retirement/Pension,No Cause Finding Issued,919,1465,0.627304,1,0.001189,30
87,Breach of Confidentiality,No Cause Finding Issued,654,1263,0.517815,1,0.001025,32
104,Constructive Discharge,No Cause Finding Issued,17858,30393,0.587570,1,0.024660,11
123,Demotion,No Cause Finding Issued,10058,17994,0.558964,1,0.014600,15
143,Discharge,No Cause Finding Issued,219563,366917,0.598400,1,0.297710,1


Export the data to Excel for further analysis.

In [30]:
writer = pd.ExcelWriter("data/query_exports/issues_by_gender.xlsx")
female_bases.to_excel(writer, "issues_by_gender_female", startcol=0, index=False)
male_bases.to_excel(writer, "issues_by_gender_male", startcol=0, index=False)
writer.save()

## What other bases accompany gender-based complaints?

Query the database to return the basis of all charges in cases where a complainant filed one or more gender-based complaints.

In [31]:
gender_discrimination_bases = pd.read_sql("""
SELECT eeoc_complaints_11_17.basis, count(*) AS charges
FROM eeoc_complaints_11_17
INNER JOIN
  (SELECT basis, charge_inquiry_seq
   FROM eeoc_complaints_11_17
   WHERE basis LIKE 'Sex%' OR basis = 'Equal Pay-Female' OR basis = 'Equal Pay-Male'
   GROUP BY eeoc_complaints_11_17.basis, eeoc_complaints_11_17.charge_inquiry_seq) gender_discrimination_bases
   ON eeoc_complaints_11_17.charge_inquiry_seq = gender_discrimination_bases.charge_inquiry_seq
GROUP BY eeoc_complaints_11_17.basis
ORDER BY eeoc_complaints_11_17.basis ASC
""",
con=conn)

In [32]:
gender_discrimination_bases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 2 columns):
basis      80 non-null object
charges    80 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.3+ KB


Let's rank the bases.

In [33]:
gender_discrimination_bases["percent_of_charges"] = gender_discrimination_bases["charges"] / gender_discrimination_bases["charges"].sum()
gender_discrimination_bases["rank"] = gender_discrimination_bases["percent_of_charges"].rank(method="min", ascending=False).astype(int)
gender_discrimination_bases.sort_values("percent_of_charges", ascending=False)

,basis,charges,percent_of_charges,rank
72,Sex-Female,397869,0.272666,1
70,Retaliation,358669,0.245802,2
74,Sex-Male,122223,0.083761,3
56,Race-Black/African American,101617,0.069640,4
0,Age,82429,0.056490,5
75,Sex-Pregnancy,81013,0.055520,6
46,Other Disability,35374,0.024242,7
76,Sex-Sexual Orientation,32399,0.022204,8
41,National Origin-Other,23245,0.015930,9
59,Race-White,21863,0.014983,10


## How do the total number of complaints measure up against a state's workforce's demographic breakdown?

Import 2016 demograhpics data on the [employment status](https://www.bls.gov/lau/ex14tables.htm) of the civilian population from the Bureau of Labor Statistics.

In [34]:
state_demos = pd.read_excel("data/2016_state_demographics.xlsx", sheet_name="for_import")
state_demos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 10 columns):
state_abbreviation                 51 non-null object
asian                              26 non-null float64
black                              39 non-null float64
hispanic_ethnicity_any_race        45 non-null float64
white                              51 non-null int64
total                              51 non-null int64
pct_asian                          51 non-null float64
pct_black                          51 non-null float64
pct_hispanic_ethnicity_any_race    51 non-null float64
pct_white                          51 non-null float64
dtypes: float64(7), int64(2), object(1)
memory usage: 4.1+ KB


In [35]:
state_demos.head()

,state_abbreviation,asian,black,hispanic_ethnicity_any_race,white,total,pct_asian,pct_black,pct_hispanic_ethnicity_any_race,pct_white
0,AK,24000.0,14000.0,24000.0,231000,335000,0.071642,0.041791,0.071642,0.689552
1,AL,NaN,520000.0,83000.0,1437000,2036000,0.000000,0.255403,0.040766,0.705796
2,AR,NaN,185000.0,94000.0,1056000,1296000,0.000000,0.142747,0.072531,0.814815
3,AZ,119000.0,159000.0,1019000.0,2658000,3076000,0.038687,0.051691,0.331274,0.864109
4,CA,3022000.0,1049000.0,6386000.0,13060000,18048000,0.167442,0.058123,0.353834,0.723626


Query the database to return the number of race-based and Hispanic national origin charges by state.

In [36]:
race_basis_charges = pd.read_sql("""
SELECT state_code,
       basis,
       count(*)
FROM eeoc_complaints_11_17
WHERE basis ILIKE 'race%'
  OR basis ILIKE '%hispanic%'
GROUP BY state_code,
         basis
ORDER BY state_code,
         basis
""",
                           con=conn)

In [37]:
race_basis_charges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 448 entries, 0 to 447
Data columns (total 3 columns):
state_code    442 non-null object
basis         448 non-null object
count         448 non-null int64
dtypes: int64(1), object(2)
memory usage: 10.6+ KB


In [38]:
race_basis_charges.head(10)

,state_code,basis,count
0,AK,National Origin-Hispanic,213
1,AK,Race-American Indian/Alaska Native,319
2,AK,Race-Asian,225
3,AK,Race-Bi-Racial/Multi-Racial,60
4,AK,Race-Black/African American,681
5,AK,Race-Native Hawaiian/Pacific Islander,62
6,AK,Race-Other - Obsolete,1
7,AK,Race-White,246
8,AL,National Origin-Hispanic,415
9,AL,Race-American Indian/Alaska Native,100


Since the BLS data only contains -- at most -- data on whites, blacks, Asians and individuals of Hispanic ethnicity (regardless of race), we can drop the rows containing charges on other bases.

In [39]:
race_basis_charges = race_basis_charges[(race_basis_charges["basis"] != "Race-American Indian/Alaska Native") &
                                        (race_basis_charges["basis"] != "Race-Asian/Pacific Islander - Obsolete") &
                                        (race_basis_charges["basis"] != "Race-Bi-Racial/Multi-Racial") &
                                        (race_basis_charges["basis"] != "Race-Native Hawaiian/Pacific Islander") &
                                        (race_basis_charges["basis"] != "Race-Other - Obsolete")]
race_basis_charges.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 233 entries, 0 to 447
Data columns (total 3 columns):
state_code    229 non-null object
basis         233 non-null object
count         233 non-null int64
dtypes: int64(1), object(2)
memory usage: 7.3+ KB


In [40]:
race_basis_charges.head()

,state_code,basis,count
0,AK,National Origin-Hispanic,213
2,AK,Race-Asian,225
4,AK,Race-Black/African American,681
7,AK,Race-White,246
8,AL,National Origin-Hispanic,415


Pivot the BLS data.

In [43]:
race_basis_charges_pivoted = race_basis_charges.pivot(index="state_code", columns="basis", values="count")
race_basis_charges_pivoted.reset_index(inplace=True)
race_basis_charges_pivoted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 5 columns):
state_code                     60 non-null object
National Origin-Hispanic       57 non-null float64
Race-Asian                     57 non-null float64
Race-Black/African American    61 non-null float64
Race-White                     58 non-null float64
dtypes: float64(4), object(1)
memory usage: 2.5+ KB


In [44]:
race_basis_charges_pivoted.head()

basis,state_code,National Origin-Hispanic,Race-Asian,Race-Black/African American,Race-White
0,NaN,4.0,6.0,35.0,16.0
1,AK,213.0,225.0,681.0,246.0
2,AL,415.0,141.0,18515.0,2176.0
3,AP,NaN,3.0,10.0,NaN
4,AR,330.0,106.0,6749.0,706.0


Join the charges data with the BLS data.

In [82]:
race_basis_charges_demos = race_basis_charges_pivoted.merge(state_demos, left_on="state_code", right_on="state_abbreviation")
race_basis_charges_demos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 50
Data columns (total 15 columns):
state_code                         51 non-null object
National Origin-Hispanic           51 non-null float64
Race-Asian                         51 non-null float64
Race-Black/African American        51 non-null float64
Race-White                         51 non-null float64
state_abbreviation                 51 non-null object
asian                              26 non-null float64
black                              39 non-null float64
hispanic_ethnicity_any_race        45 non-null float64
white                              51 non-null int64
total                              51 non-null int64
pct_asian                          51 non-null float64
pct_black                          51 non-null float64
pct_hispanic_ethnicity_any_race    51 non-null float64
pct_white                          51 non-null float64
dtypes: float64(11), int64(2), object(2)
memory usage: 6.4+ KB


In [83]:
race_basis_charges_demos.head()

,state_code,National Origin-Hispanic,Race-Asian,Race-Black/African American,Race-White,state_abbreviation,asian,black,hispanic_ethnicity_any_race,white,total,pct_asian,pct_black,pct_hispanic_ethnicity_any_race,pct_white
0,AK,213.0,225.0,681.0,246.0,AK,24000.0,14000.0,24000.0,231000,335000,0.071642,0.041791,0.071642,0.689552
1,AL,415.0,141.0,18515.0,2176.0,AL,NaN,520000.0,83000.0,1437000,2036000,0.000000,0.255403,0.040766,0.705796
2,AR,330.0,106.0,6749.0,706.0,AR,NaN,185000.0,94000.0,1056000,1296000,0.000000,0.142747,0.072531,0.814815
3,AZ,2056.0,383.0,6208.0,1328.0,AZ,119000.0,159000.0,1019000.0,2658000,3076000,0.038687,0.051691,0.331274,0.864109
4,CA,3810.0,3030.0,19962.0,4045.0,CA,3022000.0,1049000.0,6386000.0,13060000,18048000,0.167442,0.058123,0.353834,0.723626


Calculate the ratio of charges to employed individuals.

In [84]:
race_basis_charges_demos["hispanic_charges_per_thousand"] = (race_basis_charges_demos["National Origin-Hispanic"] / race_basis_charges_demos["hispanic_ethnicity_any_race"]) * 1000
race_basis_charges_demos["asian_charges_per_thousand"] = (race_basis_charges_demos["Race-Asian"] / race_basis_charges_demos["asian"]) * 1000
race_basis_charges_demos["black_charges_per_thousand"] = (race_basis_charges_demos["Race-Black/African American"] / race_basis_charges_demos["black"]) * 1000
race_basis_charges_demos["white_charges_per_thousand"] = (race_basis_charges_demos["Race-White"] / race_basis_charges_demos["white"]) * 1000
race_basis_charges_demos.rename(columns={"National Origin-Hispanic": "hispanic_charges", "Race-Asian": "asian_charges", "Race-Black/African American": "black_charges", "Race-White": "white_charges"}, inplace=True)
race_basis_charges_demos.drop("state_abbreviation", axis=1, inplace=True)
race_basis_charges_demos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 50
Data columns (total 18 columns):
state_code                         51 non-null object
hispanic_charges                   51 non-null float64
asian_charges                      51 non-null float64
black_charges                      51 non-null float64
white_charges                      51 non-null float64
asian                              26 non-null float64
black                              39 non-null float64
hispanic_ethnicity_any_race        45 non-null float64
white                              51 non-null int64
total                              51 non-null int64
pct_asian                          51 non-null float64
pct_black                          51 non-null float64
pct_hispanic_ethnicity_any_race    51 non-null float64
pct_white                          51 non-null float64
hispanic_charges_per_thousand      45 non-null float64
asian_charges_per_thousand         26 non-null float64
black_charges_per_thou

In [85]:
race_basis_charges_demos

,state_code,hispanic_charges,asian_charges,black_charges,white_charges,asian,black,hispanic_ethnicity_any_race,white,total,pct_asian,pct_black,pct_hispanic_ethnicity_any_race,pct_white,hispanic_charges_per_thousand,asian_charges_per_thousand,black_charges_per_thousand,white_charges_per_thousand
0,AK,213.0,225.0,681.0,246.0,24000.0,14000.0,24000.0,231000,335000,0.071642,0.041791,0.071642,0.689552,8.875000,9.375000,48.642857,1.064935
1,AL,415.0,141.0,18515.0,2176.0,NaN,520000.0,83000.0,1437000,2036000,0.000000,0.255403,0.040766,0.705796,5.000000,NaN,35.605769,1.514266
2,AR,330.0,106.0,6749.0,706.0,NaN,185000.0,94000.0,1056000,1296000,0.000000,0.142747,0.072531,0.814815,3.510638,NaN,36.481081,0.668561
3,AZ,2056.0,383.0,6208.0,1328.0,119000.0,159000.0,1019000.0,2658000,3076000,0.038687,0.051691,0.331274,0.864109,2.017664,3.218487,39.044025,0.499624
4,CA,3810.0,3030.0,19962.0,4045.0,3022000.0,1049000.0,6386000.0,13060000,18048000,0.167442,0.058123,0.353834,0.723626,0.596618,1.002647,19.029552,0.309724
5,CO,3013.0,534.0,6598.0,1239.0,NaN,123000.0,509000.0,2499000,2807000,0.000000,0.043819,0.181332,0.890274,5.919450,NaN,53.642276,0.495798
6,CT,1454.0,242.0,5472.0,554.0,95000.0,167000.0,267000.0,1500000,1797000,0.052866,0.092933,0.148581,0.834725,5.445693,2.547368,32.766467,0.369333
7,DC,571.0,219.0,3385.0,466.0,20000.0,128000.0,43000.0,209000,369000,0.054201,0.346883,0.116531,0.566396,13.279070,10.950000,26.445312,2.229665
8,DE,339.0,67.0,2907.0,465.0,19000.0,101000.0,48000.0,327000,453000,0.041943,0.222958,0.105960,0.721854,7.062500,3.526316,28.782178,1.422018
9,FL,10014.0,1306.0,33316.0,4949.0,283000.0,1463000.0,2488000.0,7368000,9278000,0.030502,0.157685,0.268161,0.794137,4.024920,4.614841,22.772386,0.671688


Export the data to Excel for further analysis.

In [86]:
writer = pd.ExcelWriter("data/query_exports/charges_by_state.xlsx")
race_basis_charges_demos.to_excel(writer, "charges_by_state", startcol=0, index=False)
writer.save()